In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [4]:
train_data_gen = ImageDataGenerator(rescale=1./255)
val_data_gen = ImageDataGenerator(rescale=1./255)


In [11]:
train_generator = train_data_gen.flow_from_directory(
'train',
target_size=(48,48),
batch_size=64,
color_mode="grayscale",
class_mode = "categorical")

Found 28709 images belonging to 7 classes.


In [12]:
val_generator = val_data_gen.flow_from_directory(
'test',
target_size=(48,48),
batch_size=64,
color_mode="grayscale",
class_mode = "categorical")

Found 7178 images belonging to 7 classes.


In [15]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape =(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation = 'relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation = 'softmax'))

emotion_model.compile(loss = 'categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics =['accuracy'])

In [16]:
#Trainging the neural network
emotion_model_info = emotion_model.fit_generator(
train_generator,
steps_per_epoch=28709//64,
epochs=50,
validation_data=val_generator,
validation_steps= 7178//64)

<ipython-input-16-82b1ac323e24>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 555s 1s/step - loss: 1.7999 - accuracy: 0.2590 - val_loss: 1.7152 - val_accuracy: 0.3260
Epoch 2/50
448/448 [==============================] - 194s 433ms/step - loss: 1.6271 - accuracy: 0.3639 - val_loss: 1.5251 - val_accuracy: 0.4171
Epoch 3/50
448/448 [==============================] - 147s 328ms/step - loss: 1.5207 - accuracy: 0.4145 - val_loss: 1.4465 - val_accuracy: 0.4470
Epoch 4/50
448/448 [==============================] - 263s 587ms/step - loss: 1.4480 - accuracy: 0.4448 - val_loss: 1.4018 - val_accuracy: 0.4616
Epoch 5/50
448/448 [==============================] - 292s 651ms/step - loss: 1.3919 - accuracy: 0.4674 - val_loss: 1.3465 - val_accuracy: 0.4902
Epoch 6/50
448/448 [==============================] - 157s 350ms/step - loss: 1.3454 - accuracy: 0.4887 - val_loss: 1.3054 - val_accuracy: 0.5031
Epoch 7/50
448/448 [==============================] - 160s 356ms/step - loss: 1.3013 - accuracy: 0.5052 - val_loss: 1.2799 - va

In [17]:
model_json = emotion_model.to_json()
with open("emotion_model.json",'w')as json_file:
    json_file.write(model_json)
    
#saving trained model weight
emotion_model.save_weights('emotion_model.h5')